In [ ]:
from cryosparc.tools import CryoSPARC
import json
import re
with open('/u/rposert/instance-info.json', 'r') as f:
    instance_info = json.load(f)

cs = CryoSPARC(**instance_info)
assert cs.test_connection()

project_uid = "P332"
workspace_uid = "W2"

project = cs.find_project(project_uid)

Connection succeeded to CryoSPARC API at http://cryoem0.sbi:40002


In [2]:
cs_parts_job = project.find_job('J134')
cs_parts = cs_parts_job.load_output('split_1')
rel_parts_job = project.find_job('J217')
rel_parts = rel_parts_job.load_output('imported_particles')

In [3]:
uid_pattern = r"^[0-9]{21}_"
def remove_uid(uid_mic:str) -> str:
    uid_mic = uid_mic.split('/')[-1]
    match = re.match(uid_pattern, uid_mic)
    while match:
        uid_mic = re.sub(uid_pattern, '', uid_mic)
        match = re.match(uid_pattern, uid_mic)
    return uid_mic

In [4]:
rel_parts.add_fields(fields = ['img'], dtypes = ['str'])
rel_parts['img'] = [
    f"{i}@{remove_uid(m)}"
    for m, i in zip(rel_parts['blob/path'], rel_parts['blob/idx'])
]

In [5]:
cs_parts.add_fields(fields = ['img'], dtypes = ['str'])
cs_parts['img'] = [f"{i}@{remove_uid(m)}" for m, i in zip(cs_parts['blob/path'], cs_parts['blob/idx'])]

In [6]:
for i, img in enumerate(rel_parts['img']):
    query_results = cs_parts.query({'img': img})
    assert len(query_results) == 1, 'Found multiple particles with same img name'
    rel_parts['uid'][i] = query_results['uid'][0]

In [ ]:
drop_image = rel_parts.drop_fields('img', copy = True)
cs.save_external_result(
    project_uid,
    workspace_uid,
    drop_image,
    type = 'particle',
    name = 'CryoSPARC-UIDs',
    title = 'CryoSPARC UIDs'
)   

'J6506'